In [15]:
import pandas as pd
import pydub
import os
import tqdm
from utils import get_end_from_start, get_start_end_from_file
from data import read_interview, get_features
import matplotlib.pyplot as plt
import numpy as np
import pickle
from lime import lime_tabular    
import dill

pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

# Load videos and extract features

In [ ]:
# Load videos and calculate feats
video_folder = 'pred/'
df_name = 'times.xlsx'  #Name of an Excel containing columns 'email','question','start' and 'end'

filenames = tqdm(os.listdir(video_folder))
df_startend = get_start_end_from_file(df_name)

interviews = []
for f in filenames : 
    interviews.append(read_interview(video_folder,df_startend,f))

# Tidy feats and save to csv so you don't have to recalculate everything
feats = [item for sublist in interviews for item in sublist]
feats = get_features(feats)
feats.to_csv("new_features.csv")

In [4]:
feats = pd.read_csv('new_features.csv')
feats = feats.set_index('id')

# Load saved model and explainer

In [5]:
#Lime works only with single output 

model_name = 'RF_single_output'
loaded_model = pickle.load(open(model_name, 'rb'))

with open('explainer', 'rb') as f:
    explainer = dill.load(f)

# Predict scores

In [ ]:
predictions = loaded_model.predict(feats)
predictions = [tuple(p) for p in predictions]
feats['predictions'] = predictions

# Explain results

In [19]:
# asking for explanation for LIME model

i = np.random.randint(0, feats.shape[0]) #Choose the video to explain, by default it explains a random video

exp = explainer.explain_instance(feats.iloc[i,:], loaded_model.predict_proba, top_labels=1)
exp.show_in_notebook(show_table=True, show_all=False)